In [130]:
import numpy
import random
from collections import defaultdict
from numpy import median
import urllib
import math
import random
import collections
import string
import csv
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import sklearn.metrics
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

In [131]:
def parseLabeledData(path):
    with open(path, 'r') as file:
        for line in file:
            line = line.split(',')
            yield({"asin":line[0],
                 "question":line[1],
                 "review":line[2],
                 "answer":line[3],
                 "relevance":float(line[4])
                })
        

print("Reading labeled data...")
data = list(parseLabeledData("C:/Users/Moi/Downloads/highestReviewData.csv"))
#data = parseLabeledData("/Users/Silvia/Desktop/New Data - Sheet1.csv")
asins = [d['asin'] for d in data]
queries = [d['question'] for d in data]
reviews = [d['review'] for d in data]
answers = [d['answer'] for d in data]
relevances = [d['relevance'] for d in data]
print("done")

Reading labeled data...
done


In [3]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

print("Reading all reviews & all questions...")

allReviews = parseAllReviews("C:/Users/Moi/Downloads/reviews.json")
allQuestions = parseAllQueries("C:/Users/Moi/Downloads/qa.json")

#allReviews = parseAllReviews("/Users/Silvia/Downloads/reviews.json")
#allQuestions = parseAllQueries("/Users/Silvia/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = []
for entry in allReviews.values():
    for review in entry:
        docSet.append(review["reviewText"])

for entry in allQuestions.values():
    for question in entry:
        docSet.append(question["question"])

docLen = [len(d.split()) for d in docSet]
avgdl = sum(docLen) / len(docLen)

print("done")

Reading all reviews & all questions...
done


In [4]:
def countAllWords():
    allWords = defaultdict(int)
    englishStopWords = stopwords.words('english')
    for r in allReviews.values():
        for review in r:
            review = review["reviewText"]
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            question = question["question"]
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1
    
    
    return allWords

allWords = countAllWords()

In [5]:
commonWords = sorted(allWords, key=lambda x: -allWords[x])[:5000]

In [6]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [7]:
bagCache = {}

def bagOfWords(document, length):
    if (document, length) in bagCache:
        return bagCache[(document, length)]
    
    bag = [0]*length
    
    exclude = set(string.punctuation)
    doc = ''.join(ch for ch in document if ch not in exclude)
    doc = doc.lower().split()
    
    for term in doc:
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            bag[index] = doc.count(term)
            
    bagCache[(document, length)] = bag
    
    return bag

In [251]:
def pairwiseProduct(review, question, length):
    reviewBag = bagOfWords(review, length)
    questionBag = bagOfWords(question, length)
        
    bagFeat = [0]*length
    
    for i in range(0, length):
        bagFeat[i] = reviewBag[i] * questionBag[i]
        
    #for i in range(0, length):
        #if reviewBag[i] > 0 or questionBag[i] > 0:
            #print(commonWords[i], reviewBag[i], questionBag[i], bagFeat[i])
        
    return bagFeat        

In [240]:
def featForOnlyQuestion(question, length):
    return ([1]+bagOfWords(question, length))

In [241]:
def featForQuestionAndRelevantReview(review, question, length = 1000):
    feat = [1]
    feat += pairwiseProduct(review, question, length)
    return feat

In [242]:
def normalize(featList):
    
    max = 0
    min = float('inf')
    for feat in featList:
        if feat > max: max = feat
        if feat < min: min = feat        
    
    for i in range(0,len(featList)-1):
        if (max - min) == 0: 
            max = 1
            min = 0
        featList[i] = (featList[i] - min) / (max - min)

    return featList

In [243]:
def train(X, y): 
    lr = LogisticRegression()
    lr.fit(X, y)
    
    return lr


In [244]:
def test(y, y_hat):
    #print(sklearn.metrics.r2_score(y, y_hat))
    
    
    accuracy = sklearn.metrics.accuracy_score(y, y_hat)
    precision = sklearn.metrics.precision_score(y, y_hat)
    recall = sklearn.metrics.recall_score(y, y_hat)
    
    return "{0:.2f}, {1:.2f}, {2:.2f}".format(accuracy, precision, recall)

In [245]:
def constrain(elems, point):    
    for elem in elems:
        if (elem > point): yield 1
        else: yield 0

In [648]:
random.seed(505)

thresh = 0.0

points = []

i = 0
qNum = 0
while i < len(data):
    
    qReviews = []
    qRelevances = []

    question = data[i]["question"]
    answer = data[i]["answer"]

    while (i < len(data) and data[i]["question"] == question):        
        if (data[i]["relevance"] > thresh):
            qReviews += [data[i]["review"]]
            qRelevances += [data[i]["relevance"]]
        i += 1

    if (len(qReviews) > 0):
        for qReview, qRelevance in zip(qReviews, qRelevances):
            points += [(
                featForOnlyQuestion(question, 500) + featForQuestionAndRelevantReview(qReview, question, 500), 
                1 if answer == "Y" else 0,
                qNum
            )]
    qNum += 1


keys = list(range(0, starti))
random.shuffle(keys)

X_train = [p[0] for p in points if p[2] % 2 == 0]
y_train = [p[1] for p in points if p[2] % 2 == 0]

X_test = [p[0] for p in points if p[2] % 2 == 1]
y_test = [p[1] for p in points if p[2] % 2 == 1]

print(len(X_train), len(y_train), len(X_test), len(y_test))

73796 73796 74760 74760


In [652]:
random.seed(505)

thresh = 0.3

yesPoint = [(featForQuestionAndRelevantReview(d["review"], d["question"], 500), 1 if d["answer"] == "Y" else 0, d["relevance"])
            for d, i in zip(data, range(0, len(data))) if d["answer"] == "Y" and d["relevance"] > thresh]
noPoint  = [(featForQuestionAndRelevantReview(d["review"], d["question"], 500), 1 if d["answer"] == "Y" else 0, d["relevance"])
            for d, i in zip(data, range(0, len(data))) if d["answer"] == "N" and d["relevance"] > thresh]

#yesPoint = [(featForOnlyQuestion(d["question"], 1000), 1 if d["answer"] == "Y" else 0)
#            for d in data if d["answer"] == "Y" and d["relevance"] > thresh]
#noPoint  = [(featForOnlyQuestion(d["question"], 1000), 1 if d["answer"] == "Y" else 0)
#            for d in data if d["answer"] == "N" and d["relevance"] > thresh]

#yesPoint = [(featForOnlyQuestion(d["question"], 2000)
#             + [d["relevance"]],
#             1 if d["answer"] == "Y" else 0)
#            for d in data if d["answer"] == "Y" and d["relevance"] > thresh]
#noPoint  = [(featForOnlyQuestion(d["question"], 2000)
#             + [d["relevance"]],
#             1 if d["answer"] == "Y" else 0)
#            for d in data if d["answer"] == "N" and d["relevance"] > thresh]

#yesPoint = random.sample(yesPoint, len(noPoint))

points = yesPoint + noPoint
random.shuffle(points)

X = [p[0] for p in points]
y = [p[1] for p in points]

print(len(X))

6655


In [653]:
X_train = X[:len(X)*2//3]
y_train = y[:len(X)*2//3]

X_test = X[len(X)*2//3:]
y_test = y[len(X)*2//3:]

#X_test = [featForQuestionAndRelevantReview(d["review"], d["question"]) for d in data]
#y_test = [1 if d["answer"] == "Y" else 0 for d in data]
#i_test = list(range(0, len(data)))
                                           
print(len(X_train), len(y_train), len(X_test), len(y_test))

4436 4436 2219 2219


In [654]:
lr = train(X_train, y_train)

In [655]:
print("Predict with one question and the most relevant review!")

y_hatTrain = [p[1] for p in lr.predict_proba(X_train)]
y_hatTest  = [p[1] for p in lr.predict_proba(X_test)]

#y_hatTrain = [p[1] for i, p in zip(i_train, lr.predict_proba(X_train)) if relevances[i] > thresh]
#y_hatTest  = [p[1] for i, p in zip(i_test, lr.predict_proba(X_test))  if relevances[i] > thresh]

for cutoff in range(10, 11):
    y_hatTrain_c = list(constrain(y_hatTrain, cutoff / 20))
    y_hatTest_c = list(constrain(y_hatTest, cutoff / 20))
    #print(len(y_hatTest_c), len(y_hatTest), len(X_test), len(y_test))
    train_string = test(y_train, y_hatTrain_c)
    test_string = test(y_test, y_hatTest_c)

    print(cutoff / 20, ":\t", train_string, "\t", test_string)

print()

Predict with one question and the most relevant review!
0.5 :	 0.81, 0.81, 0.99 	 0.77, 0.79, 0.97



In [657]:
confidence = [abs(y - 0.5)*2 for y in y_hatTest]
med = numpy.median(confidence)

print(med)

y_hatTest_c = list(constrain(y_hatTest, cutoff / 20))

y_test2 = [y for y, y_hat in zip(y_test, y_hatTest) if abs(y_hat-0.5)*2 > med]
y_hatTest_c2 = [y_c for y_c, y_hat in zip(y_hatTest_c, y_hatTest) if abs(y_hat-0.5)*2 > med]

print(test(y_test2, y_hatTest_c2))

0.5443199614567684
0.86, 0.87, 0.99


In [894]:
def featForQuestionAndReviews(reviews, relevances, question, length):    
    feat = [1]
    
    reviewBag = [0]*length
    
    relevances = [r for r in relevances]
    
    #totalRelevance = sum(relevances)
    
    med = numpy.median(relevances)
    
    thresh = 0.5
    count = 0
    for (review, relevance) in zip(reviews, relevances):
        if relevance >= thresh or relevance >= med:
            count += 1
    
    for (review, relevance) in zip(reviews, relevances):
        if (relevance >= thresh or relevance >= med):
            bow = bagOfWords(review, length)
            reviewBag = [x + y * (relevance) / count for x, y in zip(reviewBag, bow)]
            

    questionBag = bagOfWords(question, length)
    
    bagFeat = [0]*length
    
    for i in range(0, length):
        bagFeat[i] = questionBag[i] * reviewBag[i]
        
    cos = sum(bagFeat)
    

    feat += questionBag
    feat += [cos]
    feat += reviewBag
    
    return feat

In [895]:
random.seed(505)

thresh = 0.0

points = []

i = 0
while i < len(data):
    
    qReviews = []
    qRelevances = []

    question = data[i]["question"]
    answer = data[i]["answer"]
    
    while (i < len(data) and data[i]["question"] == question):        
        if (data[i]["relevance"] > thresh):
            qReviews += [data[i]["review"]]
            qRelevances += [data[i]["relevance"]]
        i += 1
    
    if (len(qReviews) > 0):
        featureVector = [(featForQuestionAndReviews(qReviews, qRelevances, question, 200), 1 if answer == "Y" else 0)]
        #for _ in range(0, len(qReviews)):
        points += featureVector

#yesPoint = [(featForQuestionAndReviews(d["review"], d["question"]), 1 if d["answer"] == "Y" else 0, i)
#            for d, i in zip(data, range(0, len(data))) if d["answer"] == "Y" and d["relevance"] > 0.5]
#noPoint  = [(featForQuestionAndReviews(d["review"], d["question"]), 1 if d["answer"] == "Y" else 0, i)
#            for d, i in zip(data, range(0, len(data))) if d["answer"] == "N" and d["relevance"] > 0.5]

#yesPoint = random.sample(yesPoint, len(noPoint))

random.shuffle(points)

X = [p[0] for p in points]
y = [p[1] for p in points]

print(len(X))

8280


In [896]:
X_train = X[:len(X)//2]
y_train = y[:len(X)//2]

X_test = X[len(X)//2:]
y_test = y[len(X)//2:]

print(len(X_train), len(y_train), len(X_test), len(y_test))

4140 4140 4140 4140


In [897]:
lr = train(X_train, y_train)

In [898]:
print("Predict with one question and all the most relevant review!")

y_hatTrain = [p[1] for p in lr.predict_proba(X_train)]
y_hatTest  = [p[1] for p in lr.predict_proba(X_test)]

for cutoff in range(10, 11):
    y_hatTrain_c = list(constrain(y_hatTrain, cutoff / 20))
    y_hatTest_c = list(constrain(y_hatTest, cutoff / 20))
    train_string = test(y_train, y_hatTrain_c)
    test_string = test(y_test, y_hatTest_c)

    print(cutoff / 20, ":\t", train_string, "\t", test_string)

print()

Predict with one question and all the most relevant review!
0.5 :	 0.70, 0.71, 0.97 	 0.67, 0.69, 0.94



In [901]:
confidence = [abs(y - 0.5)*2 for y in y_hatTest]
med = numpy.median(confidence)

print(med)

y_hatTest_c = list(constrain(y_hatTest, cutoff / 20))

y_test2 = [y for y, y_hat in zip(y_test, y_hatTest) if abs(y_hat-0.5)*2 > med]
y_hatTest_c2 = [y_c for y_c, y_hat in zip(y_hatTest_c, y_hatTest) if abs(y_hat-0.5)*2 > med]

print(test(y_test2, y_hatTest_c2))

y_test2 = [y for y, y_hat in zip(y_test, y_hatTest) if abs(y_hat-0.5)*2 > .9]
y_hatTest_c2 = [y_c for y_c, y_hat in zip(y_hatTest_c, y_hatTest) if abs(y_hat-0.5)*2 > .9]

print(test(y_test2, y_hatTest_c2))

0.3765919257884106
0.74, 0.75, 0.99
0.79, 0.80, 0.99
